In [1]:
import autogen
from autogen import ConversableAgent,register_function

In [8]:
config_list = [
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '******',
        'base_url':'https://api.chatanywhere.tech/v1',
        'tags': ['tool', 'gpt-3.5-turbo'],
    },
]

In [15]:
agent = ConversableAgent(
   name="chatbot",
   llm_config={"config_list": config_list,"seed":42,"temperature":0,},
   code_execution_config=False,  # 关闭代码执行.
   human_input_mode="NEVER",  # 是否需要人类输入.
)
reply = agent.generate_reply(messages=[{"content": "使用python写一个hello world程序", "role": "user"}])
print(reply)

当然，以下是一个简单的Python程序，打印出"Hello, World!":

```python
print("Hello, World!")
```


#### 创建Agent

In [16]:
from sparkai.frameworks.autogen import SparkAI
assistant=ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. You can help with simple calculations. Return 'TERMINATE' when the task is done.",
    llm_config={"config_list":config_list},
)
user_proxy=ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [17]:
def calculator(a: int, b: int):
    return a + b

In [18]:
from pydantic import BaseModel,Field
from typing_extensions import Annotated
class CalculatorParams(BaseModel):
    a: Annotated[int, Field(description="数字int类型")]
    b: Annotated[int, Field(description="数字int类型")]
    
def calculator(params: Annotated[CalculatorParams, "输入计算器."]) -> int:
    return params.a + params.b

#### 工具注册
##### 方式一

In [19]:
assistant.register_for_llm(name="calculator",description="一个简单的计算器")(calculator)
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(params: typing_extensions.Annotated[__main__.CalculatorParams, '输入计算器.']) -> int>

##### 方式二

In [20]:
register_function(
    calculator,
    caller=assistant,
    executor=user_proxy,
    name="calculator",
    description="一个简单的计算器",
)

#### 工具调用

In [21]:
chat_result=user_proxy.initiate_chat(assistant,message="10加10等于多少?")

User (to Assistant):

10加10等于多少?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_h5yFwDHEyyaaKsEBpqWgVNuq): calculator *****
Arguments: 
{"params":{"a":10,"b":10}}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_h5yFwDHEyyaaKsEBpqWgVNuq) *****
20
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

TERMINATE
综上所述，10加10等于20。

--------------------------------------------------------------------------------
